In [1]:
import torch

import requests
import json

import math
from collections import Counter

from sklearn.metrics.pairwise import cosine_similarity
from sklearn.metrics import accuracy_score

from sentence_transformers import SentenceTransformer

In [2]:
characters = ['Monica Geller', 'Joey Tribbiani', 'Chandler Bing', 'Phoebe Buffay', 'Ross Geller', 'Rachel Green', 'others']

#test set 1 = season 8

with open('sets/test_set1.json') as f:
    test_set1 = json.load(f)

with open('sets/train_set1.json') as f:
    train_set1 = json.load(f)

#CHANGE HERE
test_set = test_set1
train_set = train_set1

In [3]:
def tf_idf_calculator(lines):    
    # Compute the term frequency of each word in each sentence
    tf = []
    
    for sentence in lines:
        tf_sentence = {}
        for word in sentence.split():
            tf_sentence[word] = tf_sentence.get(word, 0) + 1
        tf.append(tf_sentence)

    # Compute the inverse document frequency of each word
    idf = {}
    for sentence in lines:
        for word in sentence.split():
            idf[word] = idf.get(word, 0) + 1
    for word in idf:
        idf[word] = math.log(len(lines) / idf[word])

    # Compute the tf-idf of each sentence
    tf_idf = []
    for i, sentence in enumerate(lines):
        tf_idf_sentence = 0
        for word in tf[i]:
            tf_idf_sentence += tf[i][word] * idf[word]
        tf_idf.append(tf_idf_sentence)
    
    return tf_idf

In [16]:
previous = 2
print(previous -1)

1


In [9]:
values = []

previous = 2
next = 2

for character in characters:

    lines_withcontext = []
    lines = []

    for idx, utterance in enumerate(train_set):
        if utterance['speakers'] != [] and character in utterance['speakers'] and idx not in [0, 1] and idx != len(train_set)- 1 and idx != len(train_set)- 2:
            lines_withcontext.append([utterance['transcript'], train_set[idx-2]['transcript'],train_set[idx-1]['transcript'], train_set[idx+1]['transcript'], train_set[idx+2]['transcript']])
            lines.append(utterance['transcript'])
        elif idx ==0:
            lines_withcontext.append([utterance['transcript'], "", "", train_set[idx+1]['transcript'], train_set[idx+2]['transcript']])
            lines.append(utterance['transcript'])
        elif idx ==1:
            lines_withcontext.append([utterance['transcript'], "", train_set[idx-1]['transcript'], train_set[idx+1]['transcript'],  train_set[idx+2]['transcript']])
            lines.append(utterance['transcript'])
        elif idx == len(train_set)-1:
            lines_withcontext.append([utterance['transcript'], train_set[idx-2]['transcript'] ,train_set[idx-1]['transcript'], "", ""])
            lines.append(utterance['transcript'])
        elif idx == len(train_set)-2:
            lines_withcontext.append([utterance['transcript'], train_set[idx-2]['transcript'] ,train_set[idx-1]['transcript'], train_set[idx+1]['transcript'], ""])
            lines.append(utterance['transcript'])
    

    tf_idf = tf_idf_calculator(lines)

    final = []

    #format: final = [...,[sentence, tf-idf score],...]
    for i,sentence in enumerate(lines):
        cell = []
        cell= [tf_idf[i], sentence]
        final.append(cell)
    
    # lower -> higher     
    sorted_list = sorted(final)

    l = []

    #get only the strings
    for elem in sorted_list:
        l.append(elem[1])

    #remove duplicates
    l = list( dict.fromkeys(l) )

    #change x for number of lines 
    values = [5,10,20,50,100,150,200,500,1000,2000,5000, 6000]

    for x in values:

        higher = l[-x:]
        lower = l[:x]

        f = open("embeddingscontext1/" + character + str(x)+ ".txt", "w")
        for line in higher:
            index = lines.index(line)

            for utterance in lines_withcontext[index]:
                f.write(utterance +"\n")
        f.close()


################################################################
#others

lines_withcontext = []
lines = []

for idx, utterance in enumerate(train_set):
    if utterance['speakers'] != [] and utterance['speakers'][0] not in characters and idx not in [0, 1] and idx != len(train_set)- 1 and idx != len(train_set)- 2:
        lines_withcontext.append([utterance['transcript'], train_set[idx-2]['transcript'],train_set[idx-1]['transcript'], train_set[idx+1]['transcript'], train_set[idx+2]['transcript']])
        lines.append(utterance['transcript'])
    elif idx ==0:
        lines_withcontext.append([utterance['transcript'], "", "", train_set[idx+1]['transcript'], train_set[idx+2]['transcript']])
        lines.append(utterance['transcript'])
    elif idx ==1:
        lines_withcontext.append([utterance['transcript'], "", train_set[idx-1]['transcript'], train_set[idx+1]['transcript'], train_set[idx+2]['transcript']])
        lines.append(utterance['transcript'])
    elif idx == len(train_set)-1:
        lines_withcontext.append([utterance['transcript'], train_set[idx-2]['transcript'] ,train_set[idx-1]['transcript'], "", ""])
        lines.append(utterance['transcript'])
    elif idx == len(train_set)-2:
        lines_withcontext.append([utterance['transcript'], train_set[idx-2]['transcript'] ,train_set[idx-1]['transcript'], train_set[idx+1]['transcript'], ""])
        lines.append(utterance['transcript'])
    
    
tf_idf = tf_idf_calculator(lines)

final = []

#format: final = [...,[sentence, tf-idf score],...]
for i,sentence in enumerate(lines):
    cell = []
    cell= [tf_idf[i], sentence]
    final.append(cell)
    
# lower -> higher     
sorted_list = sorted(final)

l = []

#get only the strings
for elem in sorted_list:
    l.append(elem[1])

#remove duplicates
l = list( dict.fromkeys(l) )

#change x for number of lines 
values = [5,10,20,50,100,150,200,500,1000,2000,5000, 6000]

for x in values:

    higher = l[-x:]
    lower = l[:x]

    f = open("embeddingscontext1/" + "others" + str(x)+ ".txt", "w")
    for line in higher:
        index = lines.index(line)

        for utterance in lines_withcontext[index]:
            f.write(utterance +"\n")
    f.close()


In [10]:
values = []


for character in characters:

    lines_withcontext = []
    lines = []

    for idx, utterance in enumerate(train_set):
        if utterance['speakers'] != [] and character in utterance['speakers'] and idx !=0:
            lines_withcontext.append([utterance['transcript'], train_set[idx-1]['transcript']])
            lines.append(utterance['transcript'])
        elif idx ==0:
            lines_withcontext.append([utterance['transcript'], ""])
            lines.append(utterance['transcript'])
    

    tf_idf = tf_idf_calculator(lines)

    final = []

    #format: final = [...,[sentence, tf-idf score],...]
    for i,sentence in enumerate(lines):
        cell = []
        cell= [tf_idf[i], sentence]
        final.append(cell)
    
    # lower -> higher     
    sorted_list = sorted(final)

    l = []

    #get only the strings
    for elem in sorted_list:
        l.append(elem[1])

    #remove duplicates
    l = list( dict.fromkeys(l) )

    #change x for number of lines 
    values = [5,10,20,50,100,150,200,500,1000,2000,5000, 6000]

    for x in values:

        higher = l[-x:]
        lower = l[:x]

        #substitute embeddings1/2

        f = open("embeddingscontext2/" + character + str(x)+ ".txt", "w")
        for line in higher:
            index = lines.index(line)

            for utterance in lines_withcontext[index]:
                f.write(utterance +"\n")
        f.close()


################################################################
#others

lines_withcontext = []
lines = []

for idx, utterance in enumerate(train_set):
    if utterance['speakers'] != [] and utterance['speakers'][0] not in characters and idx !=0:
        lines_withcontext.append([utterance['transcript'], train_set[idx-1]['transcript']])
        lines.append(utterance['transcript'])
    elif idx ==0:
        lines_withcontext.append([utterance['transcript'], ""])
        lines.append(utterance['transcript'])
    
    
tf_idf = tf_idf_calculator(lines)

final = []

#format: final = [...,[sentence, tf-idf score],...]
for i,sentence in enumerate(lines):
    cell = []
    cell= [tf_idf[i], sentence]
    final.append(cell)
    
# lower -> higher     
sorted_list = sorted(final)

l = []

#get only the strings
for elem in sorted_list:
    l.append(elem[1])

#remove duplicates
l = list( dict.fromkeys(l) )

#change x for number of lines 
values = [5,10,20,50,100,150,200,500,1000,2000,5000, 6000]

for x in values:

    higher = l[-x:]
    lower = l[:x]

    #substitute embeddings1/2

    f = open("embeddingscontext2/" + "others" + str(x)+ ".txt", "w")
    for line in higher:
        index = lines.index(line)

        for utterance in lines_withcontext[index]:
            f.write(utterance +"\n")
    f.close()


In [4]:
import numpy as np
model = SentenceTransformer('all-MiniLM-L6-v2')

def compute_embedding(previous_sentence,current_sentence):   
    
    embedding_previous = model.encode(previous_sentence)
    embedding_current = model.encode(current_sentence)

    #max
    #embedding = np.maximum(embedding_previous, embedding_current)

    #min
    #embedding = np.minimum(embedding_previous, embedding_current)

    #concatenate
    embedding = np.concatenate((embedding_previous, embedding_current), axis=0)

    # add
    #embedding = embedding_previous + embedding_current

    # mean
    #embedding = np.mean( np.array([ embedding_previous, embedding_current ]), axis=0 )

    return embedding

In [10]:
def compute_embedding2(current_sentence, previous2_sentence, previous_sentence, next_sentence, next2_sentence):
    
    embedding_previous2 = model.encode(previous2_sentence)
    embedding_previous = model.encode(previous_sentence)
    embedding_current = model.encode(current_sentence)
    embedding_next = model.encode(next_sentence)
    embedding_next2 = model.encode(next2_sentence)


    #concatenate
    embedding = np.concatenate(( embedding_previous2,embedding_previous, embedding_current, embedding_next, embedding_next2), axis=0)


    return embedding


In [15]:
sentences = []

for character in characters:
    f = open("embeddingscontext1/" + character +"5000"+ ".txt", "r")
    lines = f.readlines()
    sentences.append(lines)

'''
i=0
embeddings = {}
for character in characters:
    embeddings[character] = []
    for idx, utterance in enumerate(sentences[i]):
        if (idx % 2)==0:
            embeddings[character].append(compute_embedding(sentences[i][idx+1], utterance))

    i+=1
'''

i=0
embeddings = {}
for character in characters:
    embeddings[character] = []
    for idx in range(0, len(sentences[i]), 5):
        #current, previous, next
        embeddings[character].append(compute_embedding2(sentences[i][idx], sentences[i][idx-2],sentences[i][idx-1] ,sentences[i][idx+1], sentences[i][idx+2]))

    i+=1

#print(embeddings)

for character in characters:

    sum = 0
    for embedding in embeddings[character]:
        sum += embedding

    
    embeddings[character] = sum


KeyboardInterrupt: 

In [7]:
def compute_similarity(vector):

    angles = {}

    for character in characters:
        angles[character] = cosine_similarity(vector.reshape(1,-1), embeddings[character].reshape(1,-1))[0][0]

    
    
    #Smaller angles between vectors produce larger cosine values, indicating greater cosine similarity

    character = [i for i in angles if angles[i]==max(angles.values())]


    return character[0]

In [14]:
predicted = []
real = []

correct = 0
total = 0

for idx, utterance in enumerate(test_set):
    if len(utterance['speakers']) == 1:

        total += 1

        '''
        if idx == 0:
            line_embed = compute_embedding("", utterance['transcript'])
        else:
            line_embed = compute_embedding(test_set1[idx-1]['transcript'], utterance['transcript'])'''
        
        '''if idx == 0:
            #current, previous, next
            line_embed = compute_embedding2(utterance['transcript'], "" , test_set1[idx+1]['transcript'])
        elif idx == len(test_set)-1:
            line_embed = compute_embedding2(utterance['transcript'], test_set1[idx-1]['transcript'] , "")
        else:
            line_embed = compute_embedding2(utterance['transcript'], test_set1[idx-1]['transcript'] , test_set1[idx+1]['transcript'])'''

        '''
        if idx == 0:
            #current, previous, next
            line_embed = compute_embedding2(utterance['transcript'], "" , "", test_set1[idx+1]['transcript'])
        elif idx == 1:
            line_embed = compute_embedding2(utterance['transcript'], "" , test_set1[idx-1]['transcript'],test_set1[idx+1]['transcript'])
        elif idx == len(test_set)-1:
            line_embed = compute_embedding2(utterance['transcript'],test_set1[idx-2]['transcript'] , test_set1[idx-1]['transcript'] , "")
        else:
            line_embed = compute_embedding2(utterance['transcript'], test_set1[idx-2]['transcript'] , test_set1[idx-1]['transcript'] , test_set1[idx+1]['transcript'])'''
        
        #current, previous2, previous, next, next2
        if idx == 0:
            line_embed = compute_embedding2(utterance['transcript'], "" , "", test_set1[idx+1]['transcript'], test_set1[idx+2]['transcript'])
        elif idx == 1:
            line_embed = compute_embedding2(utterance['transcript'], "" , test_set1[idx-1]['transcript'],test_set1[idx+1]['transcript'], test_set1[idx+2]['transcript'])
        elif idx == len(test_set)-1:
            line_embed = compute_embedding2(utterance['transcript'],test_set1[idx-2]['transcript'] , test_set1[idx-1]['transcript'] , "", "")
        elif idx == len(test_set)-2:
            line_embed = compute_embedding2(utterance['transcript'],test_set1[idx-2]['transcript'] , test_set1[idx-1]['transcript'] , test_set1[idx+1]['transcript'], "")
        else:
            line_embed = compute_embedding2(utterance['transcript'], test_set1[idx-2]['transcript'] , test_set1[idx-1]['transcript'] , test_set1[idx+1]['transcript'], test_set1[idx+2]['transcript'])
    
        pred = compute_similarity(line_embed)

        predicted.append(pred)
        real.append(utterance['speakers'][0])

        if pred == utterance['speakers'][0]:
            correct+=1
        elif pred == "others" and utterance['speakers'][0] not in ['Monica Geller', 'Joey Tribbiani', 'Chandler Bing', 'Phoebe Buffay', 'Ross Geller', 'Rachel Green']:
            correct+=1

                    
print(accuracy_score(real, predicted)*100)

accuracy = (float(correct)/total)*100

print("accuracy: ", accuracy)

#print("f1 score: ", f1_score(real, predicted, average='macro'))


12.848172597005314
accuracy:  17.517308001932054
